# Problem set 2: Finding the Walras equilibrium in a multi-agent economy

[<img src="https://mybinder.org/badge_logo.svg">](https://mybinder.org/v2/gh/NumEconCopenhagen/exercises-2020/master?urlpath=lab/tree/PS2/problem_set_2.ipynb)

In [1]:
%load_ext autoreload
%autoreload 2

# Tasks

## Drawing random numbers

Replace the missing lines in the code below to get the same output as in the answer.

In [2]:
import numpy as np
np.random.seed(1986)
state = np.random.get_state()
for i in range(3):
    np.random.set_state(state)
    for j in range(2):
        x = np.random.uniform()
        print(f'({i},{j}): x = {x:.3f}')

(0,0): x = 0.569
(0,1): x = 0.077
(1,0): x = 0.569
(1,1): x = 0.077
(2,0): x = 0.569
(2,1): x = 0.077


## Find the expectated value

Find the expected value and the expected variance

$$ 
\mathbb{E}[g(x)] \approx \frac{1}{N}\sum_{i=1}^{N} g(x_i)
$$
$$ 
\mathbb{VAR}[g(x)] \approx \frac{1}{N}\sum_{i=1}^{N} \left( g(x_i) - \frac{1}{N}\sum_{i=1}^{N} g(x_i) \right)^2
$$

where $ x_i \sim \mathcal{N}(0,\sigma) $ and

$$ 
g(x,\omega)=\begin{cases}
x & \text{if }x\in[-\omega,\omega]\\
-\omega & \text{if }x<-\omega\\
\omega & \text{if }x>\omega
\end{cases} 
$$

In [3]:
# a. parameter choices
sigma = 3.14
omega = 2
N = 10000
np.random.seed(1986)

# b. draw random numbers
x = np.random.normal(loc=0, scale=sigma, size=N) # array of N draws

# c. transformation function
def g(x, omega):
    y = x.copy()
    # if x < than -omega then y is equal to -omega
    y[x < -omega] = -omega
    # if x > than omega then y is equal to omega
    y[x > omega] = omega
    return y

# d. mean and variance
mean = np.mean(g(x, omega)) # use numpy's function
var = np.var(g(x-mean, omega)) # use numpy's function, here x is deameaned for some reason
print(f'mean = {mean:.5f}, var = {var:.5f}')

mean = -0.00264, var = 2.69804


## Interactive histogram

**First task:** Consider the code below. Fill in the missing lines so the figure is plotted.

**Second task:** Create an interactive version of the figure with sliders for $\mu$ and $\sigma$.

In [4]:
# a. import
import ipywidgets as widgets
from scipy.stats import norm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# b. plotting figure
# The normal figure syntax
def fitting_normal(X, mu_guess, sigma_guess):

    # i. normal distribution from guess
    F = norm(loc=mu_guess, scale=sigma_guess)

    # ii. x-values
    x_low = F.ppf(0.001)  # x value where cdf is 0.001
    x_high = F.ppf(0.999)  # x value where cdf is 0.999
    x = np.linspace(x_low, x_high, 100)

    # iii. figure
    fig = plt.figure(dpi=100) # assign fig to be a matplotlib figure
    ax = fig.add_subplot(1, 1, 1) # create the subplot ("canvas")
    ax.plot(x, F.pdf(x), lw=2) # add the plots
    ax.hist(X, bins=100, density=True, histtype='stepfilled') # add the plots
    ax.set_ylim([0, 0.5]) # control axis
    ax.set_xlim([-6, 6]) # control axis


# c. parameters
mu_true = 2
sigma_true = 1
mu_guess = 1
sigma_guess = 2

# d. figure
X = np.random.normal(loc=mu_true, scale=sigma_true, size=10**6)

widgets.interact(fitting_normal # the figure function
                 # input in the figure function must be specified
                 , X = widgets.fixed(X) # X is fixed
                 , mu_guess=widgets.FloatSlider(description="$\mu$", min=0.1, max=5, step=0.05, value=1) # mu_guess is a slider
                 , sigma_guess=widgets.FloatSlider(description="$\sigma$", min=0.1, max=5, step=0.05, value=1) # sigma_guess is a slider
)

interactive(children=(FloatSlider(value=1.0, description='$\\mu$', max=5.0, min=0.1, step=0.05), FloatSlider(v…

<function __main__.fitting_normal(X, mu_guess, sigma_guess)>

## Modules

1. Call the function `myfun` from the module `mymodule` present in this folder.
2. Open VSCode and open the `mymodule.py`, add a new function and call it from this notebook.

In [5]:
import mymodule # import the python file mymodule - all functions inside will be callable:
mymodule.myfun(2) # calling the myfun function with input 2 - prints hello world! x 2
# I made function that takes to numbers and calculates the sum and product
_sum, _product = mymodule.quick_math(12,12) # as it returns two values to variables must be assigned
print(_sum, _product)

hello world!
hello world!
24 144


## Git

1. Try to go to your own personal GitHub main page and create a new repository. Then put your solution to this problem set in it.
2. Pair up with a fellow student. Clone each others repositories and run the code in them.

**IMPORTANT:** You will need **git** for the data project in a few needs. Better learn it know. Remember, that the teaching assistants are there to help you.

# Problem

Consider an **exchange economy** with

1. 2 goods, $(x_1,x_2)$
2. $N$ consumers indexed by $j \in \{1,2,\dots,N\}$
3. Preferences are Cobb-Douglas with truncated normally *heterogenous* coefficients

    $$
    \begin{aligned}
    u^{j}(x_{1},x_{2}) & = x_{1}^{\alpha_{j}}x_{2}^{1-\alpha_{j}}\\
     & \tilde{\alpha}_{j}\sim\mathcal{N}(\mu,\sigma)\\
     & \alpha_j = \max(\underline{\mu},\min(\overline{\mu},\tilde{\alpha}_{j}))
    \end{aligned}
    $$

4. Endowments are *heterogenous* and given by

    $$
    \begin{aligned}
    \boldsymbol{e}^{j}&=(e_{1}^{j},e_{2}^{j}) \\
     &  & e_i^j \sim f, f(x,\beta_i) =  1/\beta_i \exp(-x/\beta)
    \end{aligned}
    $$

**Problem:** Write a function to solve for the equilibrium.

You can use the following parameters:

**Hint:** The code structure is exactly the same as for the exchange economy considered in the lecture. The code for solving that exchange economy is reproduced in condensed form below.

In [6]:
# a. parameters
N = 10000
mu = 0.5
sigma = 0.2
mu_low = 0.1
mu_high = 0.9
beta1 = 1.3
beta2 = 2.1
seed = 1986

# b. draws of random numbers
np.random.seed(seed)
alphas = np.random.normal(loc=mu, scale=sigma, size=N)
alphas = np.fmax(np.fmin(alphas, mu_high), mu_low)
e1 = np.random.exponential(beta1, size=N)
e2 = np.random.exponential(beta2, size=N)

# c. demand function
def demand_good_1_func(alpha, p1, p2, e1, e2):
    I = p1*e1+p2*e2
    return alpha*I/p1 # for Cobb-Douglas preferences this is the demand - check for yourselves

# d. excess demand function
def excess_demand_good_1_func(alphas, p1, p2, e1, e2):

    # a. demand
    demand = np.sum(demand_good_1_func(alphas, p1, p2, e1, e2)) # calculate agregate demand

    # b. supply
    supply = np.sum(e1) # aggregate supply

    # c. excess demand
    excess_demand = demand-supply # excess demand - this we use for our optimization condition

    return excess_demand

# e. find equilibrium function
def find_equilibrium(alphas, p1, p2, e1, e2, kappa=0.5, eps=1e-8, maxiter=500):

    t = 0 # number of iterations
    while True: # infinite loop - run until `break` is passed - like "for i in infinity:"

        # a. step 1: excess demand
        Z1 = excess_demand_good_1_func(alphas, p1, p2, e1, e2) # excess demand is stored in Z1

        # if Z1 is close enough to our tolerance eps then break the loop -> optimum is found if excess demand is zero
        if np.abs(Z1) < eps or t >= maxiter:
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}') # print the equilibirum
            break

        # c. step 3: update p1
        p1 = p1 + kappa*Z1/alphas.size # heuristic update - we keep changing the price untill excess demand is zero

        # d. step 4: return # how far is the algorithm?
        if t < 5 or t % 25 == 0: # print the first five and each 25th iterations
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}') # print the progress
        elif t == 5:
            print('   ...')
        
        # increment t and loops starts over
        t += 1

    return p1


# f. call find equilibrium function
p1 = 1.4
p2 = 1
kappa = 0.5
eps = 1e-8
p1 = find_equilibrium(alphas, p1, p2, e1, e2, kappa=kappa, eps=eps)

  0: p1 =   1.45140633 -> excess demand ->  1028.12662815
  1: p1 =   1.48943510 -> excess demand ->   760.57530384
  2: p1 =   1.51816180 -> excess demand ->   574.53408777
  3: p1 =   1.54017076 -> excess demand ->   440.17912761
  4: p1 =   1.55720246 -> excess demand ->   340.63398830
   ...
 25: p1 =   1.62002594 -> excess demand ->     2.71044780
 50: p1 =   1.62056127 -> excess demand ->     0.00980814
 75: p1 =   1.62056320 -> excess demand ->     0.00003553
100: p1 =   1.62056321 -> excess demand ->     0.00000013
112: p1 =   1.62056321 -> excess demand ->     0.00000001


## Save and load

Consider the code below and fill in the missing lines so the code can run without any errors.

In [8]:
import pickle

# a. create some data
# create a dictionary
my_data = {}
my_data['A'] = {'a': 1, 'b': 2}
my_data['B'] = np.array([1, 2, 3])
my_data['C'] = (1, 4, 2)

# create a dictionary
my_np_data = {}
my_np_data['D'] = np.array([1, 2, 3])
my_np_data['E'] = np.zeros((5, 8))
my_np_data['F'] = np.ones((7, 3, 8))

# c. save with pickle module
# Pickle takes the data and makes it into a stream of bytes
# in this case a stream of binary bytes as 'wb' = write binary
with open(f'data.p', 'wb') as f:
    pickle.dump(my_data, f)

# d. save with numpy
np.savez(f'data.npz', **my_np_data) # can be saved with numpy

# a. try
# syntax for loading data
def load_and_print():
    with open(f'data.p', 'rb') as f:
        data = pickle.load(f)
        A = data['A']
        B = data['B']
        C = data['C']

    with np.load(f'data.npz') as data:
        D = data['D']
        E = data['E']
        F = data['F']

    print('variables loaded without error')


try:
    load_and_print()
except:
    print('an error is found')

variables loaded without error


# Extra Problems

## Multiple goods

Solve the main problem extended with multiple goods:

$$
\begin{aligned}
u^{j}(x_{1},x_{2}) & = x_{1}^{\alpha^1_{j}} \cdot x_{2}^{\alpha^2_{j}} \cdots x_{M}^{\alpha^M_{j}}\\
 &  \alpha_j = [\alpha^1_{j},\alpha^2_{j},\dots,\alpha^M_{j}] \\
 &  \log(\alpha_j) \sim \mathcal{N}(0,\Sigma) \\
\end{aligned}
$$

where $\Sigma$ is a valid covariance matrix.

In [9]:
# a. choose parameters
N = 10000
J = 3

# b. choose Sigma
Sigma_lower = np.array([[1, 0, 0], [0.5, 1, 0], [0.25, -0.5, 1]])
Sigma_upper = Sigma_lower.T
Sigma = Sigma_upper@Sigma_lower
print(Sigma)

# c. draw random numbers
alphas = np.exp(np.random.multivariate_normal(np.zeros(J), Sigma, 10000))
print(np.mean(alphas,axis=0))
print(np.corrcoef(alphas.T))

[[ 1.3125  0.375   0.25  ]
 [ 0.375   1.25   -0.5   ]
 [ 0.25   -0.5     1.    ]]
[1.89820099 1.90231575 1.64603366]
[[ 1.          0.17369157  0.11477933]
 [ 0.17369157  1.         -0.17763688]
 [ 0.11477933 -0.17763688  1.        ]]


In [14]:
# a. choose parameters
N = 10000
J = 4
p_guess = np.ones(J)
# b. choose Sigma
Sigma_lower = np.array([[1, 0, 0,0], [0.5, 1, 0,0], [0.25, -0.5, 1, 0], [0.5, -0.5, -0.2, 1]])
Sigma_upper = Sigma_lower.T
Sigma = Sigma_upper@Sigma_lower
#print(Sigma)

# c. draw random numbers
np.random.seed(3)
alphas = np.exp(np.random.multivariate_normal(np.zeros(J), Sigma, N))
#print(np.mean(alphas,axis=0))
#print(np.corrcoef(alphas.T))

# Before we created the endowments as two vectors with beta as scale:
# e1 = np.random.exponential(beta1, size=N)
# e2 = np.random.exponential(beta2, size=N)
# Now we create a matrix of endowments with a vector of betas as the scale:
beta_vec = np.random.uniform(1,3,size=J)
e_matrix = np.random.exponential(beta_vec.T, size=(N,1,J))[:,0,:]

# demand function - as before but with matrices
def demand_goods(e_matrix,alphas,p_vec):
    I = e_matrix@p_vec
    return (alphas/np.sum(alphas,axis=1)[:,np.newaxis]) * (I[:,np.newaxis]/p_vec)


supply_vec = np.sum(e_matrix,axis=0)

# calculate excess demand
def excess_demands(e_matrix,alphas,p_vec,supply_vec=supply_vec):
    total_demand = np.sum(demand_goods(e_matrix,alphas,p_vec), axis=0)
    return total_demand-supply_vec

def find_equilibrium(e_matrix,alphas,p_guess,kappa=0.5,eps=1e-8,maxiter=500):
    
    # Guess 
    p = p_guess
    t = 0
    
    # Calculate supply - Instead of one sum we now need a vector of sums
    supply_vec = np.sum(e_matrix,axis=0)
    
    while True: # same as before

        # a. step 1: excess demand
        Z = excess_demands(e_matrix,alphas,p)
        
        # b: step 2: stop?
        if  np.all(np.abs(Z[:-1]) < eps) or t >= maxiter:
            
            print(f'{t:3d}: p = {p} -> excess demand -> {Z}')
            break    
    
        # c. step 3: update p, by adjusting for excess demand for each good, plus adjusting for the last good
        p = p + kappa*Z/alphas[:,0].size - kappa*Z[-1]/alphas.size
        
        # Normalize list p to 1
        p[-1]=1
        
        # d. step 4: return 
        #if t < 5 or t%25 == 0:
            #print(f'{t:3d}: p = {p} -> excess demand -> {Z}')
        #elif t == 5:
            #print('   ...')
            
        t += 1    

    return p

# f. call find equilibrium function
p_optimal = find_equilibrium(e_matrix,alphas,p_guess)